<a href="https://colab.research.google.com/github/parsa-abbasi/NodeClassification/blob/main/NodeClassification_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Node Classification with Graph Neural Networks

- **Last updated:** April 2024
- **Repository:** [https://github.com/parsa-abbasi/NodeClassification](https://github.com/parsa-abbasi/NodeClassification)

This notebook contains all the codes within this repository and is meant to provide a step-by-step explanation of the process of creating a Graph Neural Network (GNN) for node classification.

## Graph Neural Networks (GNNs)

The general idea behind GNNs is to propagate information from the nodes' neighbors to the node itself. This is done by aggregating the information from the neighbors and updating the node's representation. This process is repeated for a number of layers, and the final representation of the node is used to do a prediction (in this case, the node's class). We can summarize the process as follows:

1. **Initialization**: Initialize the node's representation ($h_v^{(0)}$) with some information about the node (e.g., its features).
2. **Aggregation**: Aggregate the information from the node's neighbors ($\mathcal{N}_v$).
3. **Update**: Update the node's representation based on the aggregated information and the previous representation ($h_v^{(l-1)}$).
4. **Decoder**: Use the final representation of the node to do a prediction.

The aggregation and update steps are usually done using neural networks, which are trained to learn the best way to aggregate and update the information. The general formula for the update step is:

$$ h_v^{(l)} = \text{UPDATE}(h_v^{(l-1)}, \text{AGGREGATE}(\{h_u^{(l-1)}: u \in \mathcal{N}_v\})) $$

The whole topic of GNNs is how to define the $\text{AGGREGATE}$ and $\text{UPDATE}$ functions. There are many ways to do this, and the choice of these functions can greatly impact the performance of the model. We will use two fundamental types of GNN architectures in this notebook: **Graph Convolutional Network (GCN)** and **Graph Attention Network (GAT)**.

## Pytorch Geometric (PyG)

Working with graph data and implementing GNNs from scratch can be quite challenging. Fortunately, there are libraries that can help us with this task. [Pytorch Geometric (PyG)](https://pytorch-geometric.readthedocs.io/en/latest/index.html) is one of the most popular libraries in this field. It provides a set of utilities to preprocess and handle graph data, as well as a set of pre-implemented GNN layers and models. We will use PyG to implement the GNNs in this notebook.

First of all, make sure you have PyG installed. You can install it using the following command:

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00


In [2]:
# Make a list of installed packages
!pip freeze > requirements.txt

## Preparation

### Reproducibility

We will set the seed for any part of this notebook that involves randomness to ensure results reproducibility.

In [3]:
import random
import numpy as np
import os
import torch

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

SEED = 42
seed_everything(SEED)

### Device

We will also set the device to be used in this notebook. If a GPU is available, we will use it; otherwise, we will use the CPU.

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

### Dataset

We will use the **Cora** dataset for this notebook. The Cora dataset is a standard citation network dataset used for node classification tasks.
The Cora dataset consists of `2708` scientific publications classified into one of `7` classes:

- `Case_Based`
- `Genetic_Algorithms`
- `Neural_Networks`
- `Probabilistic_Methods`
- `Reinforcement_Learning`
- `Rule_Learning`
- `Theory`

Each publication in the dataset is described by a `0/1`-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of `1433` unique words. The citation network consists of `5429` links.

You can download the dataset from [here](https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz). However, the dataset is already available in the `data` directory of the repository.

**Note 1:** I assume that you are running this notebook on the [Google Colab](https://colab.research.google.com/) platform. Therefore, I wrote all the codes in a way that you just need to run the cells in order. All of the necessary data and libraries will be downloaded and installed automatically.

**Note 2:** We could also use `Planetoid` class from PyG to load the Cora dataset. However, I will load the dataset manually to show you how to work with custom datasets.

In [5]:
!wget https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz

--2024-04-19 11:59:14--  https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz
Resolving linqs-data.soe.ucsc.edu (linqs-data.soe.ucsc.edu)... 128.114.47.74
Connecting to linqs-data.soe.ucsc.edu (linqs-data.soe.ucsc.edu)|128.114.47.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168052 (164K) [application/x-gzip]
Saving to: ‘cora.tgz’

cora.tgz            100%[===================>] 164.11K   329KB/s    in 0.5s    

2024-04-19 11:59:16 (329 KB/s) - ‘cora.tgz’ saved [168052/168052]



In [6]:
!tar -xzvf cora.tgz

cora/
cora/README
cora/cora.cites
cora/cora.content


### Graph Loading

The Cora dataset is stored in two files: `cora.content` and `cora.cites`. The `cora.content` file contains the node features and labels, and the `cora.cites` file contains the citation links between the nodes. The format of the files is as follows:

- `cora.content`: `<node_id> <feature_0> <feature_1> ... <feature_n> <label>`
- `cora.cites`: `<source_node_id> <target_node_id>`

I implemented a `GraphLoader` class which can load graph data from files in the above format. The code for this class is also available in the `graph_loader.py` file in the `utils` directory of the repository.


The script will load the graph and return the following:

| Variable | Description |
| --- | --- |
| `features` | a dictionary where the key is the node ID and the value is the features of the node |
| `labels` | a dictionary where the key is the node ID and the value is the label of the node |
| `edge_index` | a list of tuples where each tuple represents an edge between two nodes |

It also provides the following methods:

| Function | Description |
| --- | --- |
| `load_data()` | Loads the graph from the given files (it will be called automatically when the object is created) |
| `get_data()` | Returns the loaded data (features, labels, and edge_index) |
| `print_info()` | Prints the information about the loaded graph (number of nodes, edges, features, and labels) |
| `to_undirected()` | Converts the graph to an undirected graph by adding the inverse edges |

In [7]:
class GraphLoader:
    '''
    `GraphLoader`: A class to load the raw graph data from the files
        - `file_nodes`: The path to the file containing the nodes data (e.g., cora.content)
            - The format of each line is <node_id> <feature_0> <feature_1> ... <feature_n> <label>
        - `file_edges`: The path to the file containing the edges data (e.g., cora.cites)
            - The format of each line is <ID of source node> <ID of target node>
    '''

    def __init__(self, file_nodes: str='', file_edges: str=''):

        # if files paths are not provided, use the default ones which is the cora dataset
        if file_nodes == '' or file_edges == '':
            data_dir = os.path.join(os.path.dirname("__file__"), 'data')
            file_nodes = os.path.join(data_dir, 'cora.content')
            file_edges = os.path.join(data_dir, 'cora.cites')
        self.file_nodes = file_nodes
        self.file_edges = file_edges

        # key: node ID, value: feature vector
        self.features = {}
        # key: node ID, value: label
        self.labels = {}
        # List of edges in the graph
        self.edge_index = []

        self.num_nodes = 0
        self.num_edges = 0
        self.num_classes = 0
        self.num_features = 0

        # Load the data from the files
        self.load_data()

    def load_data(self):

        # Load the features and labels of the nodes
        with open(self.file_nodes, 'r') as f:
            for line in f:
                line = line.strip().split()
                node = int(line[0])
                self.features[node] = list(map(int, line[1:-1]))
                self.labels[node] = line[-1]

        # Load the edges of the graph
        with open(self.file_edges, 'r') as f:
            for line in f:
                line = line.strip().split()
                self.edge_index.append([int(line[0]), int(line[1])])

        self.num_nodes = len(self.features)
        self.num_edges = len(self.edge_index)
        self.num_classes = len(set(self.labels.values()))
        self.num_features = len(self.features[list(self.features.keys())[0]])

    def get_data(self):
        return self.features, self.labels, self.edge_index

    def print_info(self):
        print('# Nodes:', self.num_nodes)
        print('# Edges:', self.num_edges)
        print('# Classes:', self.num_classes)
        print('# Features:', self.num_features)

    def to_undirected(self):
        '''
        Convert the graph to an undirected graph by adding the inverse edges (if not already exist)
        '''

        edge_index = self.edge_index
        for edge in self.edge_index:
            if [edge[1], edge[0]] not in edge_index:
                edge_index.append([edge[1], edge[0]])
        self.edge_index = edge_index
        self.num_edges = len(edge_index)

In [8]:
# from utils.graph_loader import GraphLoader

loader = GraphLoader(file_nodes='./cora/cora.content',
                     file_edges='./cora/cora.cites')
features, labels, edge_index = loader.get_data()
loader.print_info()

# Nodes: 2708
# Edges: 5429
# Classes: 7
# Features: 1433


The Cora graph is originally a directed graph, but as you can see in many publications, it is treated as an undirected graph. Therefore, we will convert the graph to an undirected graph. We can use the `to_undirected()` method of the `GraphLoader` class to do this. The method will add the inverse edges to the graph.

**Note:** The number of edges will not be exactly doubled after converting the graph to an undirected graph because the original graph may contain multiple edges between the same pair of nodes.

In [9]:
loader.to_undirected()
features, labels, edge_index = loader.get_data()
loader.print_info()

# Nodes: 2708
# Edges: 10556
# Classes: 7
# Features: 1433


### ID Mapping

The node IDs in the Cora dataset are not sequential. The graph contains `2708` nodes, but the node IDs are not in the range `[0, 2707]` (e.g., 31336, 1061127, etc.). We need to map the node IDs to sequential IDs starting from `0`. However, we need to keep the original IDs because we need it at the prediction step to produce the output in the original format.

First of all, we will make a `pd.DataFrame` from the node IDs and their labels. Then, we will use the corresponding index of the `DataFrame` as the new ID for the nodes. We will also create two dictionaries to map the original IDs to the new IDs and vice versa.

In [10]:
import pandas as pd

df = pd.DataFrame({'Node ID': list(features.keys()), 'Label': list(labels.values())})
df

,Node ID,Label
0,31336,Neural_Networks
1,1061127,Rule_Learning
2,1106406,Reinforcement_Learning
3,13195,Reinforcement_Learning
4,37879,Probabilistic_Methods
...,...,...
2703,1128975,Genetic_Algorithms
2704,1128977,Genetic_Algorithms
2705,1128978,Genetic_Algorithms
2706,117328,Case_Based


In [11]:
# id to index mapping
id_to_idx = {node_id: idx for idx, node_id in enumerate(df['Node ID'])}
# index to id mapping
idx_to_id = {idx: node_id for idx, node_id in enumerate(df['Node ID'])}

Furthermore, the edge index contains the original node IDs. We need to map these IDs to the new IDs as well. We will use `id_to_idx` dictionary to do this.

In [12]:
edge_index = np.vectorize(lambda x: id_to_idx[x])(edge_index)
edge_index

array([[ 163,  402],
       [ 163,  659],
       [ 163, 1696],
       ...,
       [2258, 1887],
       [1887, 1902],
       [1686,  837]])

### Tensor Conversion

We need to convert the features, labels, and edge index to PyTorch tensors. We will use the `torch.tensor` function to do this.

In [13]:
# features --> key: node ID, value: feature vector
# Convert features to tensor with shape (num_nodes, num_features)
features_tensor = torch.tensor([features[key] for key in features.keys()], dtype=torch.float)
features_tensor.shape

torch.Size([2708, 1433])

The labels are stored as strings in the dataset. We need to convert them to integers. We will make a dictionary to map the labels to integers. We also make a reverse dictionary to map the integers back to the labels.

In [14]:
unique_labels = set(labels.values())
unique_labels

{'Case_Based',
 'Genetic_Algorithms',
 'Neural_Networks',
 'Probabilistic_Methods',
 'Reinforcement_Learning',
 'Rule_Learning',
 'Theory'}

In [15]:
label_to_id = {label: i for i, label in enumerate(unique_labels)}
label_to_id

{'Case_Based': 0,
 'Theory': 1,
 'Rule_Learning': 2,
 'Neural_Networks': 3,
 'Probabilistic_Methods': 4,
 'Genetic_Algorithms': 5,
 'Reinforcement_Learning': 6}

In [16]:
id_to_label = {i: label for label, i in label_to_id.items()}
id_to_label

{0: 'Case_Based',
 1: 'Theory',
 2: 'Rule_Learning',
 3: 'Neural_Networks',
 4: 'Probabilistic_Methods',
 5: 'Genetic_Algorithms',
 6: 'Reinforcement_Learning'}

In [17]:
# labels --> key: node ID, value: label
# Convert labels to tensor with shape (num_nodes,)
labels_tensor = torch.tensor([label_to_id[labels[key]] for key in labels.keys()], dtype=torch.long)
labels_tensor.shape
# Uncomment the following line to convert labels to one-hot encoding (num_nodes, num_classes)
# In that case you need to change the loss function to `torch.nn.functional.cross_entropy`
# labels_one_hot = torch.nn.functional.one_hot(labels_tensor, num_classes=len(unique_labels))

torch.Size([2708])

PyG needs the edge index to be a tensor with shape `(2, num_edges)`. We need to transpose the edge index to match this shape. We also use `contiguous()` to make sure that the tensor is stored in a contiguous chunk of memory. This will be necessary for some PyTorch operations.

In [18]:
edge_index_tensor = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
edge_index_tensor.shape

torch.Size([2, 10556])

In [19]:
edge_index_tensor

tensor([[ 163,  163,  163,  ..., 2258, 1887, 1686],
        [ 402,  659, 1696,  ..., 1887, 1902,  837]])

### Data Object

The `Data` object in PyG is a data container class to store node-level and graph-level attributes. We will create a `Data` object from the features, labels, and edge index.

In [20]:
from torch_geometric.data import Data

data = Data(x=features_tensor, y=labels_tensor, edge_index=edge_index_tensor)
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708])

### Normalization

We will normalize the feature vectors in a way that the sum of the feature vectors of each node is equal to `1`. This is a common practice in GNNs to make the model invariant to the scale of the feature vectors. We will use the `NormalizeFeatures` class from PyG to do this.

In [21]:
from torch_geometric.transforms import NormalizeFeatures
data = NormalizeFeatures()(data)

In [22]:
# add the number of classes to the data object
data.num_classes = len(unique_labels)
# move the data object to the device (CPU or GPU)
data.to(device)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], num_classes=7)

## GNN Models

### Graph Convolutional Network (GCN)

The **Graph Convolutional Network (GCN)** is one of the simplest and most popular GNN architectures. The GCN layer is defined as follows:

$$ h_v^{(l+1)} = \sigma\left(\sum_{u \in \mathcal{N}_v} \frac{1}{c_{u,v}} W^{(l)} h_u^{(l)}\right) $$

where:

- $h_v^{(l)}$ is the representation of node $v$ at layer $l$.
- $\mathcal{N}_v$ is the set of neighbors of node $v$.
- $c_{u,v}$ is the product of the square root of the degrees of nodes $u$ and $v$.
- $W^{(l)}$ is the weight matrix at layer $l$.
- $\sigma$ is the activation function.

The GCN layer is implemented in PyG as `GCNConv`. We will create a simple GCN model with two GCN layers as follows:

1. **GCN Layer 1**: Input dimension is the number of features, output dimension is `hidden_channels`.
2. **Activation**: Applying an activation function (e.g., ReLU) to the output of the first GCN layer.
3. **Dropout**: Dropping out some nodes to prevent overfitting.
4. **GCN Layer 2**: Input dimension is `hidden_channels`, output dimension is the number of classes.
5. **LogSoftmax**: Producing the output in the form of log probabilities.

#### Softmax vs. LogSoftmax

Softmax is a function that takes a vector of arbitrary real-valued scores and squashes it to a vector of values between `0` and `1` that sum to `1`. It is defined as follows:

$$ \text{Softmax}(x)_i = \frac{e^{x_i}}{\sum_j e^{x_j}} $$

The problem with the softmax function is that it can easily overflow for large values of $x_i$. Therefore, it is common to use the log-softmax function, which is defined as follows:

$$ \text{LogSoftmax}(x)_i = \log\left(\frac{e^{x_i}}{\sum_j e^{x_j}}\right) = x_i - \log\left(\sum_j e^{x_j}\right) $$

In [23]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    '''
    `GCN`: A simple Graph Convolutional Network model with two layers
        - `hidden_channels`: The number of hidden channels in the GCN layers
        - `dropout`: The dropout probability
        - `activation`: The activation function to use (e.g., `torch.nn.functional.relu`)
    '''

    def __init__(self, hidden_channels, dropout=0.5, activation=F.relu):
        super(GCN, self).__init__()
        self.layer1 = GCNConv(data.num_features, hidden_channels)
        self.layer2 = GCNConv(hidden_channels, data.num_classes)
        self.dropout = dropout
        self.activation = activation


    def forward(self, data):
        # x: node features, edge_index: adjacency list
        x, edge_index = data.x, data.edge_index

        x = self.layer1(x, edge_index)
        x = self.activation(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.layer2(x, edge_index)

        return F.log_softmax(x, dim=1)

### Graph Attention Network (GAT)

The **Graph Attention Network (GAT)** is another popular GNN architecture that uses attention mechanisms to compute the importance of the neighbors of a node. The GAT layer is defined as follows:

$$ h_v^{(l+1)} = \sigma\left(\sum_{u \in \mathcal{N}_v} \alpha_{u,v} W^{(l)} h_u^{(l)}\right) $$

where:

- $\alpha_{u,v}$ is the attention coefficient between nodes $u$ and $v$.
- $W^{(l)}$ is the weight matrix at layer $l$.
- $\sigma$ is the activation function.

The attention coefficient $\alpha_{u,v}$ is computed as follows:

$$ \alpha_{u,v} = \frac{\exp\left(\text{LeakyReLU}\left(\vec{a}^T [W h_u^{(l)} || W h_v^{(l)}]\right)\right)}{\sum_{w \in \mathcal{N}_v} \exp\left(\text{LeakyReLU}\left(\vec{a}^T [W h_u^{(l)} || W h_w^{(l)}]\right)\right)} $$

where:

- $\vec{a}$ is a learnable weight vector.
- $||$ denotes concatenation.
- $\text{LeakyReLU}$ is the Leaky Rectified Linear Unit activation function.

**Multi-head attention** can be used to stabilize the learning process and improve the performance of the model. In multi-head attention, the attention mechanism is applied multiple times with different weight matrices, and the results are usually concatenated or averaged. The GAT layer in PyG supports multi-head attention.

The GAT layer is implemented in PyG as `GATConv`. We will create a simple GAT model with two GAT layers as follows:

1. **GAT Layer 1**: Input dimension is the number of features, output dimension is `hidden_channels`.
2. **Activation**: Applying an activation function (e.g., ReLU) to the output of the first GAT layer.
3. **Dropout**: Dropping out some nodes to prevent overfitting.
4. **GAT Layer 2**: Input dimension is `hidden_channels`, output dimension is the number of classes.
5. **LogSoftmax**: Producing the output in the form of log probabilities.

In [24]:
from torch_geometric.nn import GATConv

class GAT(torch.nn.Module):
    '''
    `GAT`: A simple Graph Attention Network model with two layers
        - `hidden_channels`: The number of hidden channels in the GAT layers
        - `dropout`: The dropout probability
        - `activation`: The activation function to use (e.g., `torch.nn.functional.relu`)
        - `heads`: The number of attention heads
    '''

    def __init__(self, hidden_channels, dropout=0.5, activation=F.relu, heads=8):
        super(GAT, self).__init__()
        self.layer1 = GATConv(data.num_features, hidden_channels, heads=heads, dropout=dropout)
        self.layer2 = GATConv(hidden_channels * heads, data.num_classes, heads=1, dropout=dropout)
        self.dropout = dropout
        self.activation = activation

    def forward(self, data):
        # x: node features, edge_index: adjacency list
        x, edge_index = data.x, data.edge_index

        x = self.layer1(x, edge_index)
        x = self.activation(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.layer2(x, edge_index)

        return F.log_softmax(x, dim=1)

### Graph Attention Network v2 (GATv2)

The [How Attentive are Graph Attention Networks?](https://arxiv.org/abs/2105.14491) paper demonstrates that the original GAT model has a fundamental flaw in the way it computes the attention coefficients. The original GAT model uses a static attention mechanism. They propose a new variant of the GAT model called GATv2, which uses a strictly more expressive dynamic attention. The original GAT computes the attention coefficients as follows:

$$ \vec{a}_{u,v} = \text{LeakyReLU}\left(\vec{a}^T [W h_u || W h_v]\right) $$

However, the GATv2 model computes the attention coefficients as follows:

$$ \vec{a}_{u,v} = \vec{a}^T \text{LeakyReLU}\left(W [h_u || h_v]\right) $$

The GATv2 model is implemented in PyG as `GATv2Conv`. We will create a simple GATv2 model the same as the GAT model.

In [25]:
from torch_geometric.nn import GATv2Conv

class GATv2(torch.nn.Module):
    '''
    `GATv2`: A simple Graph Attention Network model with two layers
        - `hidden_channels`: The number of hidden channels in the GAT layers
        - `dropout`: The dropout probability
        - `activation`: The activation function to use (e.g., `torch.nn.functional.relu`)
        - `heads`: The number of attention heads
    '''

    def __init__(self, hidden_channels, dropout=0.5, activation=F.relu, heads=8):
        super(GATv2, self).__init__()
        self.layer1 = GATv2Conv(data.num_features, hidden_channels, heads=heads, dropout=dropout)
        self.layer2 = GATv2Conv(hidden_channels * heads, data.num_classes, heads=1, dropout=dropout)
        self.dropout = dropout
        self.activation = activation

    def forward(self, data):
        # x: node features, edge_index: adjacency list
        x, edge_index = data.x, data.edge_index

        x = self.layer1(x, edge_index)
        x = self.activation(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.layer2(x, edge_index)

        return F.log_softmax(x, dim=1)

## Training

### Single Epoch Training

The `train` function will train the model for a single epoch. It will do the following:

1. Set the model to training mode.
2. Zero the gradients.
3. Make a forward pass by passing the `Data` object to the model.
4. Calculate the loss by comparing the output of the model with the ground truth labels for the training nodes.
5. Calculate the gradients.
6. Update the weights using the optimizer.
7. Return the loss.

We will use the negative log likelihood loss (`NLLLoss`) for this purpose because we are using log probabilities as the output of the model.

In [26]:
def train(model, optimizer, data):
    '''
    `train`: A function to train the model for one epoch
        - `model`: The model to train
        - `optimizer`: The optimizer to use
        - `data`: A PyG `Data` object containing the graph data
    '''

    # Set the model to training mode
    model.train()

    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

### Evaluation Function

The `evaluate` function will evaluate the model on the validation or test set. It will do the following:

1. Set the model to evaluation mode.
2. Make a forward pass by passing the `Data` object to the model.
3. Find the index of the class with the maximum log probability for each node.
4. Compare the predicted classes with the ground truth labels

In [28]:
from sklearn.metrics import accuracy_score

def evaluate(model, data):
    '''
    `evaluate`: A function to evaluate the model on the test set
        - `model`: The model to evaluate
        - `data`: A PyG `Data` object containing the graph data
    '''

    # Set the model to evaluation mode
    model.eval()

    out = model(data)
    pred = out.argmax(dim=1)
    pred_np = pred[data.test_mask].cpu().numpy()
    actual_np = data.y[data.test_mask].cpu().numpy()
    acc = accuracy_score(actual_np, pred_np)
    return pred_np, acc

### K-Fold Cross-Validation

K-Fold Cross-Validation is a well-known technique to fairly evaluate the performance of a model. The idea is to split the dataset into `K` folds, train the model on `K-1` folds, and test it on the remaining fold. This process is repeated `K` times, each time with a different test fold. The final performance is calculated as the average of the performance of the model on each test fold.

The `k_fold` function given below will split the dataset indices into `K` folds. It will generate two lists: one containing the training indices and the other containing the test indices for each fold.

In [29]:
from sklearn.model_selection import KFold

def k_fold(data, folds):
    '''
    `k_fold`: A function to split the data into `folds` folds using KFold
        - `data`: PyG Data object
        - `folds`: The number of folds to split the data into
    '''
    kf = KFold(n_splits=folds, shuffle=True, random_state=SEED)
    # Store indices of train and test data for each fold
    train_indices = []
    test_indices = []
    for i, (train_index, test_index) in enumerate(kf.split(data.y)):
        train_indices.append(train_index)
        test_indices.append(test_index)
    return train_indices, test_indices

### Training Loop


In [31]:
def train_k_fold(data, folds=10, epochs=100, hidden_channels=16, dropout=0.5,
                 activation=F.relu, lr=0.01, weight_decay=5e-4, layer='gcn', heads=8):
    '''
    `train_k_fold`: A function to train the model using KFold cross validation
        - `data`: PyG Data object
        - `folds`: The number of folds to split the data into
        - `epochs`: The number of epochs to train the model
        - `hidden_channels`: The number of hidden channels in the GCN layers
        - `dropout`: The dropout probability
        - `activation`: The activation function to use (e.g., `torch.nn.functional.relu`)
        - `lr`: The learning rate
        - `weight_decay`: The weight decay
        - `layer`: The type of layer to use (e.g., `gcn`, `gat`, `gatv2`)
        - `heads`: The number of attention heads (only for GAT and GATv2)
    '''

    # Split the data into folds
    train_indices, test_indices = k_fold(data, folds)

    # Store the results of each fold
    results = {'acc': []}
    predictions = []
    for i in range(folds):
        print(f"Fold {i + 1}/{folds}")
        # Create the model
        if layer == 'gcn':
            model = GCN(hidden_channels, dropout, activation).to(device)
        elif layer == 'gat':
            model = GAT(hidden_channels, dropout, activation, heads).to(device)
        elif layer =='gatv2':
            model = GATv2(hidden_channels, dropout, activation, heads).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

        # Set the train and test masks for the current fold
        data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        data.train_mask[train_indices[i]] = 1
        data.test_mask[test_indices[i]] = 1

        # Train the model
        for epoch in range(epochs):
            loss = train(model, optimizer, data)
            if epoch % 100 == 0:
                print(f"Epoch {epoch + 1}/{epochs} | Loss: {loss:.4f}")

        # Evaluate the model
        pred, acc = evaluate(model, data)
        print(f"Accuracy: {acc:.4f} | Loss: {loss:.4f}")
        print('-'*70)
        results['acc'].append(acc)
        predictions.append(pred)

    print('Overall results:')
    print(f"Average accuracy: {np.mean(results['acc']):.4f}")

    test_indices = np.concatenate(test_indices)
    predictions = np.concatenate(predictions)
    return test_indices, predictions

## Run

Now, we make a `Runner` class which helps us to set up the hyperparameters and run the training process. We will use this class to run the training process for different models and hyperparameters. This class has a `run` method that will call the `train_k_fold` method and returns the predictions.

In [32]:
class Runner:
    '''
    `Runner`: A class to run the training process
    '''

    def __init__(self, folds: int=10, epochs: int=100, hidden_channels: int=16, dropout: float=0.5,
                    activation: str='relu', lr: float=0.01, weight_decay: float=5e-4,
                    layer: str='gcn', heads: int=8):
        # Set the number of folds for KFold cross validation
        self.folds = folds
        # Set the number of epochs to train the model
        self.epochs = epochs
        # Set the number of hidden channels in the GCN/GAT layers
        self.hidden_channels = hidden_channels
        # Set the dropout probability for the output of the first layer
        self.dropout = dropout
        # Set the activation function with the pytorch implementation
        self.activation = None
        activation = activation.lower()
        if activation == 'relu':
            self.activation = F.relu
        elif activation == 'selu':
            self.activation = F.selu
        elif activation == 'elu':
            self.activation = F.elu
        elif activation == 'leaky_relu':
            self.activation = F.leaky_relu
        elif activation == 'prelu':
            self.activation = F.prelu
        else:
            raise ValueError(f"Activation function {activation} is not supported")
        # Set the learning rate for the optimizer
        self.lr = lr
        # Set the weight decay for the optimizer
        self.weight_decay = weight_decay
        # Set the layer type
        self.layer = layer.lower()
        if self.layer not in ['gcn', 'gat', 'gatv2']:
            raise ValueError(f"Layer type {self.layer} is not supported")
        self.heads = heads

    def run(self):
        test_indices, predictions = train_k_fold(data, self.folds, self.epochs, self.hidden_channels,
                                                 self.dropout, self.activation, self.lr, self.weight_decay,
                                                 self.layer, self.heads)
        return test_indices, predictions

### Experiment: GCN

We will run the training process for the GCN model and evaluate its performance.

In [33]:
gcn = Runner(folds=10, epochs=1000, hidden_channels=16, dropout=0.5,
                activation='relu', lr=0.01, weight_decay=5e-4, layer='gcn')
predictions = gcn.run()

Fold 1/10
Epoch 1/1000 | Loss: 1.9441
Epoch 101/1000 | Loss: 1.0614
Epoch 201/1000 | Loss: 0.7273
Epoch 301/1000 | Loss: 0.6147
Epoch 401/1000 | Loss: 0.5644
Epoch 501/1000 | Loss: 0.5471
Epoch 601/1000 | Loss: 0.5458
Epoch 701/1000 | Loss: 0.5270
Epoch 801/1000 | Loss: 0.5247
Epoch 901/1000 | Loss: 0.5314
Accuracy: 0.9077 | Loss: 0.5122
----------------------------------------------------------------------
Fold 2/10
Epoch 1/1000 | Loss: 1.9452
Epoch 101/1000 | Loss: 0.9547
Epoch 201/1000 | Loss: 0.6626
Epoch 301/1000 | Loss: 0.5871
Epoch 401/1000 | Loss: 0.5358
Epoch 501/1000 | Loss: 0.5172
Epoch 601/1000 | Loss: 0.5278
Epoch 701/1000 | Loss: 0.5136
Epoch 801/1000 | Loss: 0.5048
Epoch 901/1000 | Loss: 0.4923
Accuracy: 0.8745 | Loss: 0.4923
----------------------------------------------------------------------
Fold 3/10
Epoch 1/1000 | Loss: 1.9460
Epoch 101/1000 | Loss: 0.9433
Epoch 201/1000 | Loss: 0.6634
Epoch 301/1000 | Loss: 0.5943
Epoch 401/1000 | Loss: 0.5434
Epoch 501/1000 | Los

Now, we will save the predictions of the GCN model on the test set to a file (`results_gcn.tsv`).

**Note:** As we are using the K-Fold Cross-Validation technique, we will concatenate the predictions of all the folds test sets to produce the predictions on the whole graph.

In [34]:
def final_predictions(predictions: tuple) -> pd.DataFrame:
    '''
    `final_predictions`: A function to convert the predictions to a DataFrame with the original node IDs and labels
        - `predictions`: A tuple containing the test indices and the predictions
    '''
    # Unpack the predictions
    test_indices, preds = predictions
    # Convert the predictions to the original node IDs
    preds = [id_to_label[pred] for pred in preds]
    # Convert the test indices to the original node IDs
    test_indices = [idx_to_id[idx] for idx in test_indices]
    # Store the results in a DataFrame
    results = pd.DataFrame({'Node ID': test_indices, 'Prediction': preds})
    return results

In [35]:
test_indices, preds = predictions

In [36]:
results_gcn = final_predictions(predictions)
results_gcn.head()

,Node ID,Prediction
0,1106492,Theory
1,100701,Case_Based
2,237521,Neural_Networks
3,1153703,Theory
4,32872,Reinforcement_Learning


In [37]:
merged = pd.merge(df, results_gcn, on='Node ID')
acc = accuracy_score(merged['Label'], merged['Prediction'])
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.8855


In [38]:
# Save the results to a TSV file
results_gcn.to_csv('results_gcn.tsv', sep='\t', index=False)

### Experiment: GAT

We will run the training process for the GAT model and evaluate its performance.

In [39]:
gat = Runner(folds=10, epochs=1000, hidden_channels=8, dropout=0.6,
                activation='elu', lr=0.01, weight_decay=5e-4, layer='gat', heads=8)
predictions = gat.run()

Fold 1/10
Epoch 1/1000 | Loss: 1.9467
Epoch 101/1000 | Loss: 0.7969
Epoch 201/1000 | Loss: 0.7436
Epoch 301/1000 | Loss: 0.7416
Epoch 401/1000 | Loss: 0.7436
Epoch 501/1000 | Loss: 0.7285
Epoch 601/1000 | Loss: 0.7477
Epoch 701/1000 | Loss: 0.7417
Epoch 801/1000 | Loss: 0.7234
Epoch 901/1000 | Loss: 0.7377
Accuracy: 0.8819 | Loss: 0.7283
----------------------------------------------------------------------
Fold 2/10
Epoch 1/1000 | Loss: 1.9462
Epoch 101/1000 | Loss: 0.7912
Epoch 201/1000 | Loss: 0.7461
Epoch 301/1000 | Loss: 0.7384
Epoch 401/1000 | Loss: 0.7408
Epoch 501/1000 | Loss: 0.7409
Epoch 601/1000 | Loss: 0.7295
Epoch 701/1000 | Loss: 0.7325
Epoch 801/1000 | Loss: 0.7146
Epoch 901/1000 | Loss: 0.7142
Accuracy: 0.8598 | Loss: 0.7229
----------------------------------------------------------------------
Fold 3/10
Epoch 1/1000 | Loss: 1.9449
Epoch 101/1000 | Loss: 0.8281
Epoch 201/1000 | Loss: 0.7532
Epoch 301/1000 | Loss: 0.7869
Epoch 401/1000 | Loss: 0.7415
Epoch 501/1000 | Los

In [40]:
results_gat = final_predictions(predictions)
results_gat.head()

,Node ID,Prediction
0,1106492,Theory
1,100701,Case_Based
2,237521,Neural_Networks
3,1153703,Neural_Networks
4,32872,Reinforcement_Learning


In [41]:
merged = pd.merge(df, results_gat, on='Node ID')
acc = accuracy_score(merged['Label'], merged['Prediction'])
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.8874


In [42]:
# Save the results to a TSV file
results_gat.to_csv('results_gat.tsv', sep='\t', index=False)

### Experiment: GATv2

We will run the training process for the GATv2 model and evaluate its performance.

In [43]:
gatv2 = Runner(folds=10, epochs=1000, hidden_channels=8, dropout=0.6,
                activation='elu', lr=0.01, weight_decay=5e-4, layer='gatv2', heads=8)
predictions = gatv2.run()

Fold 1/10
Epoch 1/1000 | Loss: 1.9312
Epoch 101/1000 | Loss: 0.8356
Epoch 201/1000 | Loss: 0.7720
Epoch 301/1000 | Loss: 0.7483
Epoch 401/1000 | Loss: 0.7516
Epoch 501/1000 | Loss: 0.7357
Epoch 601/1000 | Loss: 0.7084
Epoch 701/1000 | Loss: 0.7227
Epoch 801/1000 | Loss: 0.7080
Epoch 901/1000 | Loss: 0.7316
Accuracy: 0.9004 | Loss: 0.7186
----------------------------------------------------------------------
Fold 2/10
Epoch 1/1000 | Loss: 1.9745
Epoch 101/1000 | Loss: 0.8483
Epoch 201/1000 | Loss: 0.7525
Epoch 301/1000 | Loss: 0.7512
Epoch 401/1000 | Loss: 0.7366
Epoch 501/1000 | Loss: 0.7073
Epoch 601/1000 | Loss: 0.7597
Epoch 701/1000 | Loss: 0.7141
Epoch 801/1000 | Loss: 0.7274
Epoch 901/1000 | Loss: 0.7142
Accuracy: 0.8561 | Loss: 0.7268
----------------------------------------------------------------------
Fold 3/10
Epoch 1/1000 | Loss: 1.9716
Epoch 101/1000 | Loss: 0.8378
Epoch 201/1000 | Loss: 0.7487
Epoch 301/1000 | Loss: 0.7578
Epoch 401/1000 | Loss: 0.7420
Epoch 501/1000 | Los

In [44]:
results_gatv2 = final_predictions(predictions)
results_gatv2.head()

,Node ID,Prediction
0,1106492,Theory
1,100701,Case_Based
2,237521,Neural_Networks
3,1153703,Theory
4,32872,Reinforcement_Learning


In [45]:
merged = pd.merge(df, results_gatv2, on='Node ID')
acc = accuracy_score(merged['Label'], merged['Prediction'])
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.8852


In [46]:
# Save the results to a TSV file
results_gatv2.to_csv('results_gatv2.tsv', sep='\t', index=False)

## Training without K-Fold Cross-Validation

In the previous experiments, we used the K-Fold Cross-Validation technique to fairly evaluate the performance of the models. Now, we want to train the models with a single train-test split to use the resulted models for interpretation purposes. First we train a GATv2 model with a single train-test split.

In [47]:
# train test split
from sklearn.model_selection import train_test_split

train_indices, test_indices = train_test_split(list(features.keys()), test_size=0.2, random_state=SEED)

# Set the train and test masks
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

train_indices = np.vectorize(lambda x: id_to_idx[x])(train_indices)
test_indices = np.vectorize(lambda x: id_to_idx[x])(test_indices)

data.train_mask[train_indices] = 1
data.test_mask[test_indices] = 1

# Create the model

model = GATv2(hidden_channels=8, dropout=0.6, activation=F.elu, heads=8).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Train the model
for epoch in range(1000):
    loss = train(model, optimizer, data)
    if epoch % 100 == 0:
        print(f"Epoch {epoch + 1}/1000 | Loss: {loss:.4f}")

# Evaluate the model
pred, acc = evaluate(model, data)
print(f"Accuracy: {acc:.4f}")

Epoch 1/1000 | Loss: 1.9574
Epoch 101/1000 | Loss: 0.8368
Epoch 201/1000 | Loss: 0.7722
Epoch 301/1000 | Loss: 0.7492
Epoch 401/1000 | Loss: 0.7226
Epoch 501/1000 | Loss: 0.7341
Epoch 601/1000 | Loss: 0.7055
Epoch 701/1000 | Loss: 0.7210
Epoch 801/1000 | Loss: 0.7358
Epoch 901/1000 | Loss: 0.7467
Accuracy: 0.8782


### Graph Visualization

We will visualize the graph interactively using the `networkx` and `plotly` libraries. The following class will help us to make a `nx` graph object from the loaded data and visualize it using the `plotly` library.

The code for this class is also available in the `graph_visualizer.py` file in the `utils` directory of the repository.

The class requires the following arguments:

- `features`: a dictionary where the key is the node ID and the value is the features of the node
- `labels`: a dictionary where the key is the node ID and the value is the label of the node
- `edge_index`: a list of tuples where each tuple represents an edge between two nodes
- `directed`: a boolean that represents whether the graph is directed or not

It provides the following methods:

| Function | Description | Arguments |
| --- | --- | --- |
| `set_node_positions())` | Sets the positions of the nodes in the graph | `pos`: a dictionary where the key is the node ID and the value is the position of the node |
| `set_layout()` | Sets the layout of the graph | `layout`: a string that represents the layout of the graph (e.g., 'spring', 'kamada_kawai', 'circular', 'random') |
| `plot()` | Plots the graph | `title`: a string, `width`: an integer, `height`: an integer |

In [49]:
import networkx as nx
import numpy as np
from plotly import graph_objects as go

class GraphVisualizer:
    def __init__(self, features: dict, labels: dict, edge_index: list, directed: bool = True):
        '''
        `GraphVisualizer`: creates a directed networkx graph from the following inputs
            - `eatures`: a dictionary where the key is the node ID and the value is the feature vector of the node
            - `labels`: a dictionary where the key is the node ID and the value is the label of the node
            - `edge_index`: a list of edges where each edge is a list [source, target]
            - `directed`: a boolean indicating if the graph is directed or not
        '''
        # Create a networkx graph
        if directed:
            self.graph = nx.DiGraph()
        else:
            self.graph = nx.Graph()
        # Add nodes and edges to the graph
        for node, feature in features.items():
            self.graph.add_node(node, feature=feature, label=labels[node])
        for edge in edge_index:
            self.graph.add_edge(edge[0], edge[1])

    def set_node_positions(self, pos: dict = None):
        '''
        `set_node_positions`: sets the positions of the nodes in the graph
            - `pos`: a dictionary where the key is the node ID and the value is the position of the node
        '''
        for node in self.graph.nodes:
            self.graph.nodes[node]['pos'] = pos[node]


    def set_layout(self, layout: str = 'spring'):
        '''
        `set_layout`: sets the layout of the graph
            - `layout`: a string indicating the layout of the graph
        '''
        if layout == 'spring':
            pos = nx.spring_layout(self.graph)
        elif layout == 'kamada_kawai':
            pos = nx.kamada_kawai_layout(self.graph)
        elif layout == 'circular':
            pos = nx.circular_layout(self.graph)
        elif layout == 'random':
            pos = nx.random_layout(self.graph)
        elif layout == 'shell':
            pos = nx.shell_layout(self.graph)
        elif layout == 'spectral':
            pos = nx.spectral_layout(self.graph)
        elif layout == 'planar':
            pos = nx.planar_layout(self.graph)
        elif layout == 'fruchterman_reingold':
            pos = nx.fruchterman_reingold_layout(self.graph)
        else:
            raise ValueError('Invalid layout')

        self.set_node_positions(pos)

    def plot(self, title: str = 'Graph Visualization', width: int = 1000, height: int = 1000):
        '''
        `plot`: plots the graph using plotly
        '''

        # Set the node positions if they are not already set
        if 'pos' not in self.graph.nodes[list(self.graph.nodes.keys())[0]]:
            self.set_layout(layout='spring')

        # Get the node positions
        node_x = []
        node_y = []
        for node in self.graph.nodes():
            x, y = self.graph.nodes[node]['pos']
            node_x.append(x)
            node_y.append(y)

        # Get the node labels and set the colors
        labels = [self.graph.nodes[node]['label'] for node in self.graph.nodes()]
        idx = np.unique(labels, return_inverse=True)[1]
        colors = idx
        # Get the number of neighbors for each node
        num_neighbors = [len(list(nx.all_neighbors(self.graph, node))) for node in self.graph.nodes()]

        # Create the node trace
        node_trace = go.Scatter(
            x = node_x, y = node_y,
            mode = 'markers',
            hoverinfo = 'text',
            hovertext = [f'Label: {label}<br>Number of neighbors: {num}' for label, num in zip(labels, num_neighbors)],
            marker = dict(
                size = 12,
                line_width = 2,
                colorscale = 'Portland',
                color = colors,
            ),
        )

        # Get the edge positions
        edge_x = []
        edge_y = []
        for edge in self.graph.edges():
            x0, y0 = self.graph.nodes[edge[0]]['pos']
            x1, y1 = self.graph.nodes[edge[1]]['pos']
            edge_x.append(x0)
            edge_x.append(x1)
            edge_x.append(None)
            edge_y.append(y0)
            edge_y.append(y1)
            edge_y.append(None)

        # Create the edge trace
        edge_trace = go.Scatter(
            x = edge_x, y = edge_y,
            line = dict(width=0.5, color='#888'),
            hoverinfo = 'none',
            mode = 'lines')

        # Create the figure
        fig = go.Figure(data=[edge_trace, node_trace],
                        layout = go.Layout(
                        width = width,
                        height = height,
                        title = title,
                        titlefont_size = 16,
                        showlegend = False,
                        hovermode = 'closest',
                        margin = dict(b=20,l=5,r=5,t=40),
                        xaxis = dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis = dict(showgrid=False, zeroline=False, showticklabels=False))
                        )
        fig.show()

### Embedding Visualization

We will visualize the node embeddings produced by the GATv2 model using the `t-SNE` algorithm. The `t-SNE` algorithm is a dimensionality reduction technique that can reduce the dimensionality of the data while preserving the local structure of the data points. We will use this technique to reduce the dimensionality of the node embeddings to `2` dimensions so that we can visualize them.

#### First Layer

First, we will visualize the node embeddings produced by the first GATv2 layer.

In [66]:
from sklearn.manifold import TSNE

# Set the model to evaluation mode
model.eval()
# Pass the data to the first layer of the model
embeddings_l1 = model.layer1(data.x, data.edge_index).detach().cpu().numpy()

# Apply t-SNE to reduce the dimensionality of the embeddings to 2D
tsne = TSNE(n_components=2)
embeddings_l1_2d = tsne.fit_transform(embeddings_l1)

In [67]:
v_features, v_labels, v_edge_index = loader.get_data()
positions = {idx_to_id[i]: embeddings_l1_2d[i] for i in range(len(embeddings_l1_2d))}
graph_visualizer = GraphVisualizer(v_features, v_labels, v_edge_index, directed=True)
graph_visualizer.set_node_positions(positions)
graph_visualizer.plot(title='Embeddings of the first layer', width=1000, height=800)

#### Second Layer

Now, we will visualize the node embeddings produced by the second GATv2 layer which is the final representation of the nodes.

In [ ]:
# Set the model to evaluation mode
model.eval()
# Extract the embeddings from the second layer
embeddings_l2 = model.layer1(data.x, data.edge_index)
embeddings_l2 = model.activation(embeddings_l2)
embeddings_l2 = F.dropout(embeddings_l2, p=0.6, training=model.training)
embeddings_l2 = model.layer2(embeddings_l2, data.edge_index).detach().cpu().numpy()

# Apply t-SNE to reduce the dimensionality of the embeddings to 2D
tsne = TSNE(n_components=2)
embeddings_l2_2d = tsne.fit_transform(embeddings_l2)

In [69]:
positions = {idx_to_id[i]: embeddings_l2_2d[i] for i in range(len(embeddings_l2_2d))}
graph_visualizer = GraphVisualizer(v_features, v_labels, v_edge_index, directed=True)
graph_visualizer.set_node_positions(positions)
graph_visualizer.plot(title='Embeddings of the second layer', width=1000, height=800)